# 🔍 Objective:

This project will introduce you to Retrieval Augmented Generation and how it can be used to expand the knowledge base of an existing pretrained LLM.

# 🧠Glossary:

We're going to be using some rather fancy sounding words that you may not have come across before. I'd highly reccommend googling them, but feel free to refer to this cheat sheet if you forget.


1.   Vector: Think of this as a list of numbers specifically used to represent co-ordinates. So a vector containing the co-ordinates to a point at (2,8) would be [2, 8]. Now we'll often be using arrays of vectors, we'll also be using a library called numpy, which supports 2D arrays (basically an array of arrays) much better than regular Python.

2.   Embeddings:

*   A huge part of RAG is semantic search (i.e. searching by meaning). This is the core of what makes RAG so powerful, as it can tell us how similar two sentences are based on what they mean, even if they are worded differently.

*   The way this is done is by using an embedding model to convert text (or images) to points in space. The closer two points are, the closer in meaning their corresponding texts are.

*   We will be storing the co-ordinates of the points in vectors, hence the need for a vector database.

*   Note: While it is helpful to think of the points generated by an embedding model as points in 3 dimensions, most embedding models generate points in higher dimensions; the model we are using generates points in 384 dimensions! So instead of our points having an x, y and z co-ordinate, they will have x, y, z, w, v,..... co-odinates.

3. LLM: Large Language Models (LLM) can generate text based on a provided prompt. Sound familiar? It should; ChatGPT is a Large Language Model! We'll be using an LLM to actually answer a users question. The problem here is that LLMs aren't all knowing; they can only answer questions based on what they've been trained on. A way to remedy this is to use techniques like RAG to work out what snippets of text from an external source are most similar to the users question and feeding them to the LLM along with the users question. The LLM can use this data to generate a natural sounding answer.

4. Chunks:
* A chunk is simply a smaller piece of a larger piece of text. The reason you'd want to break down, say, a book or a pdf is to find relevant pieces of information in it. After all, we don't want to throw an entire PDF at our LLM and have it decipher all of it, now do we?

* We'll use a bunch of functions that we'll get to later to work out which chunks are the most relevant to our question and pass just those to our LLM.

* The size of a chunk is entirely up to the programmer, but you want a chunk to be big enough for it to actually have some sort of meaning in it on its own, while not being big enought to contain too many different kinds of information in it.


# 📌 What You'll Do:



1.   Create a rudimentary vector database by defining the functions in the template class below
2.   Convert a pdf of a stock trading tutorial to a bunch of small strings (called chunks. we'll get to why later).
3.   Use this vector database to store the embeddings of the chunks.
4. Handle user queries by querying our database for the most relevant chunks and feeding them to our LLM to generate an output.

In [4]:
#library installations. we'll be using sentence_transformers for our embedding model and pypdf to read a pdf of our choosing.

%pip install pypdf
%pip install sentence_transformers

#fun fact: the ! is used to signify that these are shell commands

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 302 kB 3.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 340 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 66.5 MB 62.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 32.3 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 37.3 MB/s eta 0:00:01
     |████████████████████████████████| 11.1 MB 38.8 MB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 109.5 MB/s eta 0:00:01
     |████████████████████████████████| 481 kB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 12.0 MB/s eta 0:00:01
     |███████████████████

In [2]:
import numpy
import transformers
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader as PDFReader
from transformers import pipeline
from huggingface_hub import login

#I highly recommend sticking to this LLM; its small and runs decent in a colab notebook, especially if you have GPU acceleration enabled.
LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#Same goes for our embedding model.
EMBEDDING_MODEL = "all-MiniLM-L12-v2"
HF_API_KEY = "YOUR-HF-API-KEY-HERE"#Not a necessity.

/Users/khaita/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/khaita/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
login(token=HF_API_KEY)

In [4]:
class VectorDB:
  embedModel = None#we will be using embedModel to refer to our embedding model through out this class. Its value is initialized in the constructor

  nDim = 0#number of dimensions. Remember how I told you the embedding model generates points in space?
          #nDim will contain the number of dimensions of the space these points are generated in. It will come in handy later

  _embeddings = None#This is the numpy array that will actually store our embeddings. the underscore at the start of the variable name
                    #signifies that this a private variable; i.e it should not be directly accessed outside of this class.

  _strings : list[str] = []#we will also be storing the strings that we are generating embeddings for.


  #note: _embeddings is a numpy array. THIS IS NOT THE SAME AS A REGULAR PYTHON ARRAY.
  #      _strings on the other hand is a regular ole Python array.



  def __init__(self, model_name : str) -> None:#DO NOT CHANGE. This is the constructor funtion.
    #we've set up a lot of the boilerplate code for you; this code instantiates all the class variables and loads the embedding model.
    self.embedModel = SentenceTransformer(model_name)
    self.nDim = self.embedModel.get_sentence_embedding_dimension()

    self._embeddings = numpy.ndarray((0, self.nDim), dtype = numpy.float32) #sets _embeddings to an array of an array.
    #The first number (0) represents the number of arrays that are stored in _embeddings. 0 for now as its empty.
    #nDim refers to the number of values in each of those subarrays. We need one to represent each dimension, the same way you'd need an array with 3 values to represent a 3d point.

    self._strings = []

  def addToDatabase(self, input : list[str]):
    '''
    Your code should do two things here:
    1. Convert the strings in the input array to embeddings and add them to the _embeddings array
    2. Store their corresponding strings (IN THE SAME ORDER) in the _strings array

    Hint: the documentation for the SentenceTransformers library can be found here: https://sbert.net/docs/package_reference/sentence_transformer/index.html
    If you want to know which function to call for anything related to the embedding model (eg: generating embeddings) have a look at this.
    '''
    #Your code goes here!

    #The pass command will cause everything within this function thats after pass to be ignored.
    #Delete it once you start coding or keep all your code above it
    # Generate embeddings for the input strings
    new_embeddings = self.embedModel.encode(input)

    # Add the new embeddings and strings to the existing arrays
    if self._embeddings.size == 0:
      self._embeddings = new_embeddings
      self._strings = input
    else:
      self._embeddings = numpy.concatenate((self._embeddings, new_embeddings), axis=0)
      self._strings.extend(input)

  def clearDatabase(self):
    '''
    This function should clear the database by emptying the _embeddings and _string arrays.
    '''
    #Your code goes here!

    # Reset embeddings and strings arrays
    self._embeddings = numpy.ndarray((0, self.nDim), dtype=numpy.float32)
    self._strings = []

  def euclideanSim(self, x, y, dimensions: int):
    '''
    This function calculates how close two points are using euclidean distance.

    Euclidean distance isn't anything fancy; it's the most basic method for comparing the distance between two points.
    You may have seen it being used like this: √((x2 - x1)²  +  (y2 - y1)²) for measuring distances in 2-D.

    Your function should do the same thing, but in nDim dimensions instead.

    Keep in mind that this function is meant to return similarity i.e the opposite of distance. (This isn't mandatory; just rename the function to avoid confusion if you'd rather just have it return distances)
    After all, if two points are close to each other, their respective texts must be similar in meaning.
    '''
    #Your code goes here!

    # Calculate Euclidean distance between two points
    distance = numpy.sqrt(numpy.sum((x - y) ** 2))

    # Return similarity (opposite of distance)
    return 1 / (1 + distance)

  def search(self, input : str, n_return = 1):
    '''
    This is the biggest function here by far.
    This functions job is to find the n closest to the input in our database

    This will be done by generating an embedding for our input, and finding the n closest points to it and what pieces of text are associated to them.

    It should return a tuple (tRText, tRSim).
    tRText will contain the n closest pieces of text and tRSim will contain their respective similarities to the query
    '''
    tRText = []
    tRSim = numpy.array([])

    #Your code goes here!

    tRText = []
    tRSim = numpy.array([])
    
    # Generate embedding for the input
    query_embedding = self.embedModel.encode(input)

    # Calculate similarities with all stored embeddings
    similarities = []
    for i in range(len(self._strings)):
        sim = self.euclideanSim(query_embedding, self._embeddings[i], self.nDim)
        similarities.append(sim)
    
    # Get indices of n highest similarities
    top_indices = numpy.argsort(similarities)[-n_return:][::-1]
    
    # Get the strings and similarities for the top matches
    tRText = [self._strings[i] for i in top_indices]
    tRSim = numpy.array(similarities)[top_indices]

    return(tRText, tRSim)


In [5]:
vDB = VectorDB(model_name = EMBEDDING_MODEL)
#this code calls the constructor and sets vDB to be an instance of our vector database.


In [6]:
def chunksFromText(text : str):
  '''
  Your goal for this function is to convert the text of a page to individual chunks and store these chunks in our vector database.

  Q: What do CHUNK_SIZE and CHUNK_OVERLAP mean?

  A: CHUNK_SIZE is just the number of characters we are storing in each chunk. CHUNK_OVERLAP refers to the number of characters
  shared between sequential chunks. I.E for a chunk overlap of 50 the first chunk will store store characters 0 to 500, the second 450 to 950,
  the third 900 to 1400 and so on. Feel free to play around with these values; I chose them at random.

  Also note that chunking by character count isn't the only approach that you can use. Another method is to have each chunk be a sentence,
  multiple sentences, or an entire paragraph. Again, feel free to experiment!
  '''

  CHUNK_SIZE = 500
  CHUNK_OVERLAP = 50

  docChunks = []

  #Your code goes here!
  
  CHUNK_SIZE = 500
  CHUNK_OVERLAP = 50
  
  docChunks = []
  start = 0
  
  while start < len(text):
      end = min(start + CHUNK_SIZE, len(text))
      chunk = text[start:end].strip()
      if chunk:
          docChunks.append(chunk)
      start = end - CHUNK_OVERLAP
  
  vDB.addToDatabase(input = docChunks)

In [7]:
def chunksFromPDF(path = "", startPage = 0, endPage = None):
  '''
  Here's the documentation for the PDFReader library: https://pdfreader.readthedocs.io/en/latest/tutorial.html
  Your goal for this function is to extract text from each page in the PDF and call chunksFromText on the text to convert it to chunks and store
  it in the vector database.

  Quick reference: pdf = PDFReader(path) #gets us a a PDFReader object that we can call functions on
                   pdf.pages #big array containing all the pages
                   page = pdf.pages[0] #gets first page
                   text = page.extract_text() #gets text from the page

  You are required to clean the text from each page, as well as decide which pages are actually relevant.
  Examples of things you should get rid of (Not exhaustive):
  1. Headers, footers,
  2. Indexes,
  3. Title page,
  etc.
  '''
  #Your code goes here!

  pdf = PDFReader("/Users/khaita/Documents/PSU/NAiSS MLB/Spring 2025/project-3-rag-chatbot-khai-ta/Understanding Stocks.pdf")
    
  if endPage is None:
      endPage = len(pdf.pages)
  
  for i in range(startPage, endPage):
      page = pdf.pages[i]
      text = page.extract_text()
      
      # Basic text cleaning
      text = text.strip()
      # Remove common headers/footers
      text = text.replace("Page", "").replace("©", "").replace("All rights reserved", "")
      # Remove multiple spaces
      text = ' '.join(text.split())
      
      if text:  # Only process non-empty pages
          chunksFromText(text)

In [8]:
answerBot = pipeline("text-generation", model = LLM_MODEL, trust_remote_code=True) # This bit of code loads our LLM into memory; we can access it using
                                                                                   # the answerBot object.
                                                                                   #The model we are using is 2.2GB, so make sure you've got a decent WiFi connection.

Device set to use mps:0


In [26]:
def generateAnswer(question):
  '''
  Documentation for the pipeline object (answerBot) we created two lines ago: https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/pipelines#transformers.TextGenerationPipeline

  This is it, the final piece of the puzzle. This function will take a question, find out which chunks from our vector database
  are most relevant to it, and ask our LLM the original question with the context of the text from the relevant chunks.

  Have it return a string with the LLMs answer.
  '''
  #Your code goes here!

  # Get relevant chunks from the database
  relevant_chunks, _ = vDB.search(question, n_return=5)
    
  # Create context from relevant chunks
  context = "\n".join(relevant_chunks)
    
  # Create prompt with context and question
  prompt = f"""Based on the following context, please answer the question. If the answer cannot be found in the context, say "I don't have enough information to answer that question."

Context:
{context}

Question: {question}

Answer:"""
    
# Generate answer using the LLM
  output = answerBot(
      prompt, 
      max_length=500, 
      do_sample=True, 
      temperature=0.7,
      truncation=True,
      num_return_sequences=1
  )[0]['generated_text']
  
  # Extract just the answer part and clean it
  answer = output.split("Answer:")[-1].strip()

  # Remove any additional questions or prompts
  answer = answer.split("Question:")[0].strip()

  return answer

In [27]:
print(generateAnswer("What is a 401k?"))

A 401k is a retirement savings plan offered by many companies, typically through their paycheck deduction. It allows employees to set aside a portion of their paycheck, typically every paycheck, and then receive matching funds from the company. The funds can be used to invest in stocks, bonds, or other investment vehicles. The employee also receives a tax break for the contributions made to the 401k.


📌 Wrapping Up

And that should be it! Try calling the generate answer function we just defined with a finance related question.

I'd recommend asking it what a 401(k) is. The PDF we used uses a fancy bit of unicode to write 59.5 (which is how old you have to be before you start paying taxes on a 401k), and if our code is working properly, the LLM will work that character into its answer.

I hope you had fun working on this project and learned something new